In [ ]:
%load_ext autoreload
%autoreload 2


import numpy as np
import time, os, sys

import skimage.io
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300


import delta
# delta.assets.download_assets()

In [ ]:
"""
This script runs the segmentation U-Net
For mother machine data, it runs on images of cropped out and resized single
chambers as fed to it in Pipeline processing.

The images are processed by batches of 4096 to prevent memory issues.

@author: jblugagne
"""

import os
import glob

import numpy as np

from delta.data import saveResult_seg, predictGenerator_seg, postprocess, readreshape
from delta.model import unet_seg
import delta.utilities as utils
from delta.utilities import cfg

# Load config:
cfg.load_config(presets="2D")
# cfg.load_config(presets="mothermachine")


# Input image sequence (change to whatever images sequence you want to evaluate):
inputs_folder = '/home/kcutler/DataDrive/omnipose_all/small/test'

# # For mother machine instead:
# cfg.load_config(presets='mothermachine')

# # Images sequence (change to whatever images sequence you want to evaluate):
# inputs_folder = os.path.join(cfg.eval_movie,'cropped_rois')

# Outputs folder:
outputs_folder = os.path.join(inputs_folder, "DeLTA")
if not os.path.exists(outputs_folder):
    os.makedirs(outputs_folder)

# List files in inputs folder:
unprocessed = sorted(
    glob.glob(inputs_folder + "/*img.tif")
)

# Load up model:
model = unet_seg(input_size=cfg.target_size_seg + (1,))
model.load_weights(cfg.model_file_seg)

# Process
while unprocessed:
    # Pop out filenames
    ps = min(4096, len(unprocessed))  # 4096 at a time
    to_process = unprocessed[0:ps]
    del unprocessed[0:ps]

    # Input data generator:
    predGene = predictGenerator_seg(
        inputs_folder,
        files_list=to_process,
        target_size=cfg.target_size_seg,
        crop=cfg.crop_windows,
    )

    # mother machine: Don't crop images into windows
    if not cfg.crop_windows:
        # Predictions:
        results = model.predict(predGene, verbose=1)[:, :, :, 0]

    # 2D: Cut into overlapping windows
    else:
        img = readreshape(
            os.path.join(inputs_folder, to_process[0]),
            target_size=cfg.target_size_seg,
            crop=True,
        )
        # Create array to store predictions
        results = np.zeros((len(to_process), img.shape[0], img.shape[1], 1)) #bad, assumes the same shape everywhere ?
        print(img.shape)
        
        
        # Crop, segment, stitch and store predictions in results
        for i in range(len(to_process)):
            # Crop each frame into overlapping windows:
            windows, loc_y, loc_x = utils.create_windows(
                next(predGene)[0, :, :], target_size=cfg.target_size_seg
            )
            # We have to play around with tensor dimensions to conform to
            # tensorflow's functions:
            windows = windows[:, :, :, np.newaxis]
            # Predictions:
            pred = model.predict(windows, verbose=1, steps=windows.shape[0])
            print(pred.shape)
            # Stich prediction frames back together:
            pred = utils.stitch_pic(pred[:, :, :, 0], loc_y, loc_x)
            print(pred.shape)
            pred = pred[np.newaxis, :, :, np.newaxis]  # Mess around with dims

            results[i] = pred

    # Post process results (binarize + light morphology-based cleaning):
    results = postprocess(results, crop=cfg.crop_windows)

    # Save to disk:
    saveResult_seg(outputs_folder, results, files_list=to_process)

In [ ]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import cv2
from IPython import display

files_folder = '/home/kcutler/DataDrive/omnipose_all/small/test'
images = glob.glob(files_folder+'/*img.tif')
images.sort()

# Load config:
cfg.load_config(presets='2D')
savefile = cfg.model_file_seg

# Load weights from file:
model.load_weights(savefile)

for filename in images:
    
    # Read image:
    I = cv2.imread(filename, cv2.IMREAD_ANYDEPTH)
    # Rescale to 0-1:
    I = (I.astype(np.float32)-np.min(I))/np.ptp(I)
    # Run prediction:
    y = model.predict(I[np.newaxis,:,:,np.newaxis])
    # Display output:
    plt.imshow(y[0,:,:,0])
    display.display(plt.gcf())
    display.clear_output(wait=True)

In [ ]:
import os

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from delta.utilities import cfg
from delta.model import unet_seg
from delta.data import trainGenerator_seg

# Load config:
cfg.load_config(presets='2D')

# Files:
# Point to extracted folder:
training_set = 'home/kcutler/DataDrive/omnipose_all/train_sorted_website'

# Overwrite seg model:
savefile = cfg.model_file_seg

# Training parameters:
batch_size = 4
epochs = 600
steps_per_epoch = 300
patience = 50

#Data generator parameters:
data_gen_args = dict(
    rotation = 2,
    rotations_90d = True,
    zoom=.15,
    horizontal_flip=True,
    vertical_flip=True,
    histogram_voodoo=True,
    illumination_voodoo=True,
    gaussian_noise = 0.03,
    gaussian_blur = 1
    )

# Generator init:
myGene = trainGenerator_seg(
    batch_size,
    os.path.join(training_set,'img'),
    os.path.join(training_set,'seg'),
    os.path.join(training_set,'wei'),
    augment_params = data_gen_args,
    target_size = cfg.target_size_seg,
    crop_windows = cfg.crop_windows
    )

# Define model:
model = unet_seg(input_size = cfg.target_size_seg+(1,))
model.summary()

# Callbacks:
model_checkpoint = ModelCheckpoint(
    savefile, monitor='loss', verbose=2, save_best_only=True
    )
early_stopping = EarlyStopping(
    monitor='loss', mode='min', verbose=2, patience=patience
    )

# Train:
history = model.fit(
    myGene,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    callbacks=[model_checkpoint, early_stopping]
    )

In [ ]:
import h5py

In [ ]:
h5py.File('/home/kcutler/DataDrive/DeLTA/delta/assets/models/unet_pads_track.hdf5')